# Big example for Radix-based Benders

In [1]:
%load_ext line_profiler

In [2]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f1fb7bf8d88>)


### Try optimizing using radix for one condition first

In [3]:
#----------------------------------------
# Starting from basal model
ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')

In [4]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

In [5]:
print mu_crowd0

0.972916210772


In [6]:
opt = Optimizer(ijomc)
gap = opt.add_duality_gap_constraint(INF=1e3)

In [7]:
gap.optimize(solver='gurobi')

<Solution 1.95 at 0x7f1f541c4a50>

In [8]:
import numpy as np

radix = 2.
powers = np.arange(-2,3)
print(powers)
digits_per_power = radix
pwr_max = max(powers)
digits = np.linspace(0, radix-1, digits_per_power)
print(digits)

# Discretize crowding coefficients into radix
crowding_p = gap.metabolites.crowding

var_cons_dict = {}
for rxn_p in crowding_p.reactions:
    # Get the coefficient in the dual
    var_d = gap.reactions.wa_crowding
    cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]
    a0 = rxn_p.metabolites[crowding_p]    
    var_cons_dict[rxn_p.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

[-2 -1  0  1  2]
[0. 1.]


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [9]:
len(var_cons_dict)

2018

In [10]:
%lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

In [11]:
# %time opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

In [12]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary'):
    rxn.objective_coefficient = 1.

In [13]:
N_CONDS = 1

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r.id for r in ijomc.reactions.query('EX_')]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [14]:
mu_meas = 0.74

F_TOL = 0.005

mu_id = 'BIOMASS_Ec_iJO1366_core_53p95M'
rxn_mu = gap.reactions.get_by_id(mu_id)
rxn_mu.lower_bound = mu_meas*(1-F_TOL)
rxn_mu.upper_bound = mu_meas*(1+F_TOL)

from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

In [15]:
milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 167579 rows, 73010 columns and 418477 nonzeros
Variable types: 52830 continuous, 20180 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-12, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+03]
  RHS range        [5e-03, 1e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 54171 rows and 8552 columns
Presolve time: 3.62s
Presolved: 113408 rows, 64458 columns, 299255 nonzeros
Variable types: 44278 continuous, 20180 integer (20180 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.339641e+05   9.877939e+09      6s
   12681    4.7946657e-04   5.031057e-04   1.537201e+05     10s
   13437    1.9786869e-03   2.342049e-04   2.213424e+06     15s
   14004    2.0291434e-03   2.342552e-04   2.213424e+06     

In [16]:
sol = gurobi_solver.format_solution(milp, gap)
muopt = sol.x_dict[rxn_mu.id]
yopt = [sol.x_dict[rxn.id] for rxn in gap.reactions.query('binary_')]
sum(yopt)
print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))
print('Number of non-zero binaries: %g' % sum(yopt))

Initial mu_crowd=0.972916. Fitted within 0.5%: 0.7363 <= 0.741869 <= 0.7437
Number of non-zero binaries: 8


In [17]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol.x_dict[yid]
            if abs(y)>1e-10:
                print yid, y
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

binary_CYTBO3_4pp14 1.0
binary_ATPS4rpp_abs13 1.0
binary_ATPS4rpp_abs14 1.0
binary_MDH_abs14 1.0
binary_PGK_abs14 1.0
binary_G6PDH2r_abs14 1.0
binary_NH4tpp_abs13 1.0
binary_NH4tpp_abs14 1.0


In [20]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

[(u'CYTBO3_4pp', 0.00010366907685298552),
 (u'MDH_abs', 5.055667931392883e-05),
 (u'G6PDH2r_abs', 4.918465665512613e-05),
 (u'ATPS4rpp_abs', 4.1567317560316426e-05),
 (u'PGK_abs', 0.0006039610048946835),
 (u'NH4tpp_abs', 0.00014751285544170527)]

### Plug back in to be sure

In [19]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding_bound = ijomc.metabolites.crowding._bound
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

KeyError: u'CSND'

In [ ]:
ijofit.optimize()